# Marketing Data Warehouse & BI Analytics Platform
## Exploratory Data Analysis & Predictive Modeling

This notebook connects to the marketing data warehouse, computes key KPIs, explores campaign and creative performance, and trains a simple predictive model for conversions.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy as sa

# Display options
pd.set_option("display.max_columns", 100)

In [3]:
# Connect to SQLite DW (DB is one level up from notebook folder)
engine = sa.create_engine("sqlite:////mnt/data/marketing_dw.db")

fact = pd.read_sql("SELECT * FROM fact_marketing_performance", engine)
dim_platforms = pd.read_sql("SELECT * FROM dim_platform", engine)
dim_campaigns = pd.read_sql("SELECT * FROM dim_campaign", engine)
dim_adset = pd.read_sql("SELECT * FROM dim_adset", engine)
dim_creatives = pd.read_sql("SELECT * FROM dim_creative", engine)
dim_countries = pd.read_sql("SELECT * FROM dim_country", engine)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [4]:
# Build a Unified DataFrame
df = (
        fact
        .merge(dim_platforms, on="platform_key")
        .merge(dim_campaigns, on="campaign_key")
        .merge(dim_adset, on="adset_key")
        .merge(dim_creatives, on="creative_key")
        .merge(dim_countries, on="country_key")
    )

df.head()

NameError: name 'fact' is not defined

In [ ]:
# KPI Calculations
df["ctr"] = df["clicks"] / df["impressions"].replace(0, np.nan)
df["cpc"] = df["spend"] / df["clicks"].replace(0, np.nan)
df["cpm"] = df["spend"] / (df["impressions"].replace(0, np.nan) / 1000)
df["cvr"] = df["conversions"] / df["clicks"].replace(0, np.nan)
df["roas"] = df["revenue"] / df["spend"].replace(0, np.nan)
df["cac"] = df["spend"] / df["new_customers"].replace(0, np.nan)

df.describe()[["impressions", "clicks", "spend", "conversions", "revenue"]]

## Campaign Performance Overview

In [ ]:
campaign_perf = (
    df.groupby(["platform_name", "campaign_name"], as_index=False)
      .agg({
          "impressions": "sum",
          "clicks": "sum",
          "spend": "sum",
          "conversions": "sum",
          "revenue": "sum",
          "new_customers": "sum"
      })
)
campaign_perf["ctr"] = campaign_perf["clicks"] / campaign_perf["impressions"]
campaign_perf["cvr"] = campaign_perf["conversions"] / campaign_perf["clicks"].replace(0, np.nan)
campaign_perf["roas"] = campaign_perf["revenue"] / campaign_perf["spend"].replace(0, np.nan)
campaign_perf["cac"] = campaign_perf["spend"] / campaign_perf["new_customers"].replace(0, np.nan)

campaign_perf.sort_values("roas", ascending=False)


In [1]:
# Creative Leaderboard & Scatter
creative_perf = (
    df.groupby(["platform_name", "creative_name"], as_index=False)
      .agg({
          "impressions": "sum",
          "clicks": "sum",
          "spend": "sum",
          "conversions": "sum",
          "revenue": "sum"
      })
)
creative_perf["ctr"] = creative_perf["clicks"] / creative_perf["impressions"].replace(0, np.nan)
creative_perf["cvr"] = creative_perf["conversions"] / creative_perf["clicks"].replace(0, np.nan)
creative_perf["roas"] = creative_perf["revenue"] / creative_perf["spend"].replace(0, np.nan)

creative_perf.sort_values("roas", ascending=False).head(10)


NameError: name 'df' is not defined

In [ ]:
# Scatter
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=creative_perf,
    x="ctr",
    y="roas",
    size="spend",
    hue="platform_name"
)
plt.title("CTR vs ROAS by Creative")
plt.xlabel("CTR")
plt.ylabel("ROAS")
plt.tight_layout()
plt.show()


In [ ]:
# Simple Predictive Model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

features = df[["impressions", "clicks", "spend"]]
target = df["conversions"]

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.3, random_state=42
)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

pd.Series(model.feature_importances_, index=features.columns).sort_values(ascending=False)


In [ ]:
## Key Insights Summary

- **Best-performing platform**: (Fill based on ROAS results)
- **Top campaigns**: (List top 3 by ROAS)
- **Top creatives**: (List top 3 by ROAS or CVR)
- **Main driver of conversions**: (Based on feature importance)
- **Next optimization ideas**:
  - Pause poor ROAS campaigns
  - Increase budget on creatives with high CTR + ROAS
  - Test new hooks for low CTR creatives


## Top Creatives by ROAS

In [ ]:
df.groupby("creative_name")["roas"].mean().sort_values(ascending=False).head(10)

## CTR vs ROAS Scatter Plot

In [ ]:
sns.scatterplot(data=df, x="ctr", y="roas", hue="platform_name", size="spend")
plt.title("CTR vs ROAS — Creative Efficiency")
plt.show()

## Simple Predictive Model (Conversions)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

features = df[["impressions", "clicks", "spend"]]
target = df["conversions"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
print("MAE:", mean_absolute_error(y_test, pred))

pd.Series(model.feature_importances_, index=features.columns)